In [677]:
    import twitter
    import re 
    from selenium import webdriver
    from bs4 import BeautifulSoup
    import time 
    import pandas as pd 
    from webdriver_manager.chrome import ChromeDriverManager
    import datetime as dt
    import os
    import tweepy as tw
    import numpy as np
    import logging

# TTFL Data

### Everyday Data

In [662]:
url_ttfl = 'https://fantasy.trashtalk.co/login/#login'
email = ''
mdp = ''

In [672]:
%%time
def get_soup():
    op = webdriver.ChromeOptions()
    op.add_argument('headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(),options = op)
    driver.get(url_ttfl)
    time.sleep(0.3)

    #Check if there are asking for data management 

    data_button = driver.find_element_by_class_name('sd-cmp-JnaLO')
    if data_button is not None :
        data_button.click()
    driver.find_element_by_name('email').send_keys(email)
    driver.find_element_by_name('password').send_keys(mdp)
    driver.find_element_by_xpath("//button[@class='btn btn-primary font-montserrat all-caps fs-12 pull-right xs-pull-left']").click()
    driver.find_element_by_css_selector('[href = "/?tpl=team"] ').click()
    src = driver.page_source 
    soup = BeautifulSoup(src,'html.parser')
    driver.close()
    return soup 



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [/Users/alexandretaranoff/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


lalala
CPU times: user 94.8 ms, sys: 28.1 ms, total: 123 ms
Wall time: 12.3 s


In [751]:
%%time

"""This will get your daily data and returns the appropriate data frame and dict.
I append all of the attributes in a dictionnary. Could have been done in classes I know. 
        
                                        """
def get_data(soup):
    ranking = []
    twitter_name = []
    name = []
    total_points = []
    last_night_points = []
    last_night_player = []
    evolution = []
    data = soup.find('table',{'data-plugin' : 'dataTable'})
    table = data.find_all('tr',{'role' : 'row'})[1:]
    for rows in table :
        all_rows = rows.find_all('td')
        ranking.append(int(all_rows[0].get_text(strip = True)))
        twitter_link = all_rows[1].find('img',{})
        if twitter_link is not None : 
            twitter_name.append(twitter_link["data-original-title"])
            name.append(all_rows[1].find_all('a',{})[1].get_text(strip = True))
        else : 
            twitter_name.append(None)
            name.append(all_rows[1].find_all('a',{})[0].get_text(strip = True))
        total_points.append(all_rows[2].get_text(strip = True))
        evolution.append(all_rows[3].get_text(strip = True ))
        points_name = all_rows[4].get_text(strip = True)
        print(points_name)
        regex_number = re.compile(r'(\d)+')
        print(regex_number.search(points_name).group())
        last_night_points.append(int(regex_number.search(points_name).group()))
        regex_name = re.compile(r'(\w(\w)+\s(\w)+)|(\w\.\w\.\s(\w)+)')
        if regex_name.search(points_name) != None :
            last_night_player.append(regex_name.search(points_name).group())
        else : last_night_player.append(None)
        date = [dt.datetime.today().strftime('%d-%m-%Y')]*len(ranking)
        dic_ttfl_caviste = {'ranking' : ranking, 'name' : name ,  'twitter_name' :  twitter_name,
                            'total_points' : total_points, 'last_night_player' : last_night_player ,
                            'last_night_points' : last_night_points,'date_pick': date}
        data_day = pd.DataFrame(dic_ttfl_caviste, index = dic_ttfl_caviste['date_pick']).sort_values(by =['ranking','name'], ascending = True).drop(columns= ['date_pick'])

    return data_day,dic_ttfl_caviste



CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


### Monthly  & Deck précédent 

In [691]:
%%time

"""This will get your daily data and returns the appropriate data frame and dict.
I append all of the attributes in a dictionnary. Could have been done in classes I know.  """
def get_data_deck_month(soup):
    points = []
    name = []
    html_deck = soup.find('div',{'id': 'exampleTopComponents'}).find('tbody',{}).find_all('tr',{})
    html_month = soup.find('div',{'id': 'exampleTopCss'}).find('tbody',{}).find_all('tr',{})
    month = []
    deck_number = []
    number_str = soup.find('a',{'href': '#exampleTopComponents'}).get_text(strip = True)
    regex_number = re.compile(r'(\d)+')
    number = int(regex_number.search(number_str).group())

    for html in html_month : 
        html_m = html.find_all('td',{})
        twitter_or_not = html_m[1].find_all('a')
        name.append(twitter_or_not[len(twitter_or_not)-1].get_text(strip = True))
        points.append(int(html_m[2].get_text(strip = True)))
        month.append(dt.datetime.today().month)

    name_2 = []
    points_2 =[]
    for html in html_deck : 
        html_m = html.find_all('td',{})
        twitter_or_not = html_m[1].find_all('a')
        name_2.append(twitter_or_not[len(twitter_or_not)-1].get_text(strip = True))
        points_2.append(int(html_m[2].get_text(strip = True)))
        deck_number.append(number)
    
    dic_month = {'name' : name, 'points' : points, 'month' : month }
    dic_deck = {'name' : name_2 , 'points' : points_2, 'deck_number' : deck_number }
    data_month = pd.DataFrame(dic_month).sort_values(by = ['points','month'], ascending = False)
    data_deck = pd.DataFrame(dic_deck).sort_values(by=['points','deck_number'],ascending = False )
    return data_month,data_deck,dic_month,dic_deck

CPU times: user 12 µs, sys: 41 µs, total: 53 µs
Wall time: 55.8 µs


### Twitter API 

In [692]:
api_key = 'Your Api Key'
api_secret_key = 'Your Secret API Key'
access_token = 'Your access token'
access_secret_token = 'Your secret access token'

In [693]:
auth = tw.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_secret_token)
api = tw.API(auth, wait_on_rate_limit=True)

In [694]:
'''Get the tweet with the scores'''


def get_tweet_daily(dic_day):
    nb_joueurs = len(dic_day["name"])
    somme = 0 
    if dic_day['last_night_player'] == [None]*nb_joueurs:
        tweet = ' Pas de match hier soir, on se revoit demain pour une update en cas de match'
        tweet_2 = ''
    else :
        tweet = 'On est le ' + str(dic_day["date_pick"][0]) + ' et  le bilan de la veille est : \n'
        tweet_2 = ''
        tweet_temp = tweet
        name = ''
        for i in range(nb_joueurs):
            somme += dic_day["last_night_points"][i]
            if dic_day["twitter_name"][i] is not None :
                name = '@'+ str(dic_day["twitter_name"][i])
            else : 
                name = dic_day["name"][i]
            tweet_temp += '- {} : {} pour {} pts \n'.format(name,dic_day['last_night_player'][i],
                                                                 dic_day["last_night_points"][i])
            if len(tweet_temp) > 270:
                tweet_2 += '-{} : {} pour {} pts \n'.format(name,dic_day['last_night_player'][i],
                                                                 dic_day["last_night_points"][i])
            else : 
                tweet += '-{} : {} pour {} pts \n'.format(name,dic_day['last_night_player'][i],
                                                                 dic_day["last_night_points"][i])
    return tweet,tweet_2,somme 



In [696]:
"""Get the gif that you will send along with the daily tweet"""

def get_gif_daily(somme):
    if somme == 0 : 
        choix = gifs_happy[np.random.randint(len(gifs_happy))]
    elif somme < 200 : 
        choix = gifs_not_happy[np.random.randint(len(gifs_not_happy))]
    elif somme < 400 :
        choix = gifs_meh[np.random.randint(len(gifs_meh))]
    else : 
        choix = gifs_happy[np.random.randint(len(gifs_happy))]
    return choix 

In [641]:
%%time
'''Get the tweet you will send each week'''
def get_tweet_deck(dic_deck,data_day):
    tweet = 'On est lundi, petit récap du deck n°{} \n'.format(dic_deck['deck_number'][0])
    nb_joueurs = len(dic_deck["deck_number"])
    for i in range(nb_joueurs):
        name = data_day.loc[data_day['name']== dic_deck["name"][i],["twitter_name"]]["twitter_name"][0]
        if name == None : 
            name = dic_deck["name"][i]
        tweet+= "{}.@{}:{}pts\n".format(i+1,name,dic_deck['points'][i])
        if i == 0 :
            tweet_best = "Pour le deck n°{}, notre meilleur joueur est @{} 🥳🥳🥳🥳🥳".format(dic_deck['deck_number'][0],
                                                                                        name)
    return tweet,tweet_best

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 8.82 µs


In [660]:
'''Put all the gifs that you are gonna be using in your tweets'''

gifs_happy = []


gifs_not_happy = []


gifs_monday = []

gif_month = []


In [670]:
def main():
    soup = get_soup()
    data_day,dic_day = get_data(soup)
    data_daily_his = pd.read_excel('Your File address', index_col = 0)
    data_daily_upd = pd.concat([data_daily_his,data_day])
    auth = tw.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_secret_token)
    api = tw.API(auth, wait_on_rate_limit=True)
    tweet_day,tweet_2_day,somme = get_tweet_daily(dic_day)
    print('You have your first tweet & gif')
    gif_day = get_gif_daily(somme)
    if dt.datetime.today().weekday() == 0 : 
        data_month,data_deck,dic_month,dic_deck = get_data_deck_month(soup)
        data_month_his = pd.read_excel('Your File address', index_col = 0)
        data_deck_his = pd.read_excel('Your File address', index_col = 0)
        data_deck_upd = pd.concat([data_deck_his,data_deck])
        tweet,tweet_best = get_tweet_deck(dic_deck,data_day)
        gif = gifs_monday[np.random.randint(len(gifs_monday))]
        status = api.update_with_media(gif,tweet)
        status = api.update_status(tweet_best)
        data_deck_upd.to_excel('your file address')  
    print("Almost Done")
    data_daily_upd.to_excel('Your File address') 
    status = api.update_with_media(gif_day,tweet_day) 
    if tweet_2_day != '':
        status = api.update_status(tweet_2_day)

In [657]:
%%time
main()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [/Users/alexandretaranoff/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


/Users/alexandretaranoff/Desktop/TTFL/dwight_monday.gif
CPU times: user 244 ms, sys: 70.1 ms, total: 314 ms
Wall time: 25.2 s


In [583]:
"""Gifs Links and names in order to downloads them on your computer"""


gif_links = []

gif_names = []

In [584]:

#for i in range(len(gif_links)):
    #os.system("curl {} --output ~/Desktop/TTFL/gif_ttfl/{}.gif".format(gif_links[i],gif_names[i]))
    
    